In [10]:
import pandas as pd
import sys
import os
from pathlib import Path
import sqlite3

project_root = os.path.abspath('../../')
if project_root not in sys.path:
    sys.path.append(project_root)
    
from src.data.db import DBConnection, execute_query, get_pitcher_data
from config import DBConfig

In [26]:
class NotebookDBConnection(DBConnection):
    def __init__(self):
        db_path = os.path.join(project_root, 'data', 'pitcher_stats.db')
        super().__init__(db_name=db_path)

In [20]:
pd.set_option('display.max_columns', 150)

In [27]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print([table[0] for table in tables])

['pitcher_ids', 'sqlite_sequence', 'teams', 'statcast_pitchers']


In [28]:
with NotebookDBConnection() as conn:
    df_loaded = pd.read_sql_query("SELECT * FROM statcast_pitchers", conn)
    

In [29]:
df_loaded.head()

,id,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,pitcher_id,season
0,1,FS,2024-09-29,87.9,-2.23,5.63,"Eovaldi, Nathan",683021,543135,single,hit_into_play,None,None,None,None,8.0,"Angels challenged (tag play), call on the fiel...",R,R,R,LAA,TEX,X,7.0,line_drive,1,1,2024,-1.21,0.09,0.08,1.95,NaN,642136.0,NaN,2,7,Bot,62.91,109.50,None,None,None,None,8.155121,-127.840544,-3.075595,-14.889497,22.532697,-30.742560,3.29,1.53,169.0,99.6,8.0,88.8,1671.0,6.5,746174,641680,663993,543760,677649,669701,694671,665750,608671,53.98,0.649,0.617,0.9,1.0,1.0,0.0,4.0,56,3,Split-Finger,0,3,0,3,3,0,0,3,Standard,Standard,238.0,-0.031,0.700,66.2,7.0,0.763,-0.700,99.6,-3,-3,0.096,0.096,34,28,34,28,3,2,5.0,1.0,NaN,NaN,2.82,1.21,1.21,33.5,543135,2024
1,2,FC,2024-09-29,87.5,-2.18,5.71,"Eovaldi, Nathan",683021,543135,None,ball,None,None,None,None,12.0,"Angels challenged (tag play), call on the fiel...",R,R,R,LAA,TEX,B,NaN,None,0,1,2024,0.37,0.50,1.12,2.74,NaN,642136.0,NaN,2,7,Bot,NaN,NaN,None,None,None,None,7.268350,-127.388330,-2.165408,2.810565,22.327468,-26.475579,3.29,1.51,NaN,NaN,NaN,88.8,2251.0,6.7,746174,641680,663993,543760,677649,669701,694671,665750,608671,53.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,2,Cutter,0,3,0,3,3,0,0,3,Standard,Standard,171.0,0.000,0.012,NaN,NaN,NaN,-0.012,NaN,-3,-3,0.096,0.096,34,28,34,28,3,2,5.0,1.0,NaN,NaN,2.44,-0.37,-0.37,36.7,543135,2024
2,3,FF,2024-09-29,94.7,-2.30,5.43,"Eovaldi, Nathan",683021,543135,None,foul,None,None,None,None,2.0,"Angels challenged (tag play), call on the fiel...",R,R,R,LAA,TEX,S,NaN,None,0,0,2024,-0.80,0.97,-0.15,3.04,NaN,642136.0,NaN,2,7,Bot,NaN,NaN,None,None,None,None,7.460556,-137.739717,-2.768912,-11.783645,29.171384,-19.404262,3.29,1.53,163.0,74.0,63.0,95.5,2087.0,6.7,746174,641680,663993,543760,677649,669701,694671,665750,608671,53.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,1,4-Seam Fastball,0,3,0,3,3,0,0,3,Standard,Standard,228.0,0.000,-0.040,64.0,6.2,NaN,0.040,88.0,-3,-3,0.096,0.096,34,28,34,28,3,2,5.0,1.0,NaN,NaN,1.57,0.80,0.80,30.0,543135,2024
3,4,FF,2024-09-29,94.1,-2.36,5.45,"Eovaldi, Nathan",694359,543135,field_out,hit_into_play,None,None,None,None,5.0,Niko Kavadas flies out to left fielder Wyatt L...,R,L,R,LAA,TEX,X,7.0,fly_ball,0,1,2024,-1.29,1.00,0.13,2.53,NaN,642136.0,NaN,1,7,Bot,51.13,113.29,None,None,None,None,9.356835,-136.799712,-4.126632,-18.213832,28.567894,-18.831807,3.27,1.60,281.0,96.2,51.0,94.8,2253.0,6.6,746174,641680,663993,543760,677649,669701,694671,665750,608671,53.85,0.006,0.006,0.0,1.0,0.0,0.0,3.0,55,2,4-Seam Fastball,0,3,0,3,3,0,0,3,Infield shade,Standard,228.0,-0.038,-0.226,NaN,NaN,0.008,0.226,96.2,-3,-3,

In [24]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    cursor.execute("PRAGMA table_info(statcast_pitcher)")
    print(cursor.fetchall())
    

[(0, 'pitch_type', 'TEXT', 0, None, 0), (1, 'game_date', 'TEXT', 0, None, 0), (2, 'release_speed', 'REAL', 0, None, 0), (3, 'release_pos_x', 'REAL', 0, None, 0), (4, 'release_pos_z', 'REAL', 0, None, 0), (5, 'player_name', 'TEXT', 0, None, 0), (6, 'batter', 'INTEGER', 0, None, 0), (7, 'pitcher', 'INTEGER', 0, None, 0), (8, 'events', 'TEXT', 0, None, 0), (9, 'description', 'TEXT', 0, None, 0), (10, 'spin_dir', 'INTEGER', 0, None, 0), (11, 'spin_rate_deprecated', 'INTEGER', 0, None, 0), (12, 'break_angle_deprecated', 'INTEGER', 0, None, 0), (13, 'break_length_deprecated', 'INTEGER', 0, None, 0), (14, 'zone', 'INTEGER', 0, None, 0), (15, 'des', 'TEXT', 0, None, 0), (16, 'game_type', 'TEXT', 0, None, 0), (17, 'stand', 'TEXT', 0, None, 0), (18, 'p_throws', 'TEXT', 0, None, 0), (19, 'home_team', 'TEXT', 0, None, 0), (20, 'away_team', 'TEXT', 0, None, 0), (21, 'type', 'TEXT', 0, None, 0), (22, 'hit_location', 'INTEGER', 0, None, 0), (23, 'bb_type', 'TEXT', 0, None, 0), (24, 'balls', 'INTEGER'